# Document Creation for NLP Tasks
This notebook loads the Google reviews data and creates two distinct documents for each review:
1. **Review Document:** The raw text of the customer review.
2. **Business Document:** A concatenation of the business name, category, and description.

In [1]:
import pandas as pd

## 1. Load the data

In [3]:
df = pd.read_csv('../data_gpt_labeler/final_data_sampled.csv')

In [5]:
df.head()

,Unnamed: 0,rating,text,business_name,business_category,business_description,_id
0,848717,5,great for sunset,'Ehukai Beach Park,"['Park', 'Public beach', 'Tourist attraction']",Popular surfing beach offering massive wintert...,1.1426748616291053e+20_1589655333572
1,848703,5,The main access point to popular surfing break...,'Ehukai Beach Park,"['Park', 'Public beach', 'Tourist attraction']",Popular surfing beach offering massive wintert...,1.1447491719186307e+20_1591852064598
2,848713,5,Great place to set up your camping chairs and ...,'Ehukai Beach Park,"['Park', 'Public beach', 'Tourist attraction']",Popular surfing beach offering massive wintert...,1.0893375045945795e+20_1463461472578
3,848706,5,My favorite Beach for surfing on Oahu North Sh...,'Ehukai Beach Park,"['Park', 'Public beach', 'Tourist attraction']",Popular surfing beach offering massive wintert...,1.1249899958787118e+20_1570685676722
4,848694,5,Excellent beach for family activities great su...,'Ehukai Beach Park,"['Park', 'Public beach', 'Tourist attraction']",Popular surfing beach offering massive wintert...,1.1730942640485394e+20_1605375558437


## 2. Create Review and Business Documents

In [8]:
df['review_document'] = df['text'].astype(str)
df['business_document'] = df['business_name'].fillna('') + ' ' + df['business_category'].fillna('') + ' ' + df['business_description'].fillna('')
df[['review_document', 'business_document']].head()

,review_document,business_document
0,great for sunset,"'Ehukai Beach Park ['Park', 'Public beach', 'T..."
1,The main access point to popular surfing break...,"'Ehukai Beach Park ['Park', 'Public beach', 'T..."
2,Great place to set up your camping chairs and ...,"'Ehukai Beach Park ['Park', 'Public beach', 'T..."
3,My favorite Beach for surfing on Oahu North Sh...,"'Ehukai Beach Park ['Park', 'Public beach', 'T..."
4,Excellent beach for family activities great su...,"'Ehukai Beach Park ['Park', 'Public beach', 'T..."


## 3. Display a single example

In [11]:
print('Review Document:')
print(df['review_document'].iloc[0])
print('---')
print('Business Document:')
print(df['business_document'].iloc[0])

Review Document:
great for sunset
---
Business Document:
'Ehukai Beach Park ['Park', 'Public beach', 'Tourist attraction'] Popular surfing beach offering massive wintertime waves, with calmer waters in the summer.


## 4. Create the Corpus

In [14]:
corpus = []
for index, row in df.iterrows():
    corpus.append(row['review_document'])     
    corpus.append(row['business_document']) 
print(f'Total number of documents in the corpus: {len(corpus)}')
print('---')
print('First 4 documents in the corpus:')
for doc in corpus[:4]:
    print(doc)    
print('---')

Total number of documents in the corpus: 20000
---
First 4 documents in the corpus:
great for sunset
'Ehukai Beach Park ['Park', 'Public beach', 'Tourist attraction'] Popular surfing beach offering massive wintertime waves, with calmer waters in the summer.
The main access point to popular surfing breaks such as Pipeline.
'Ehukai Beach Park ['Park', 'Public beach', 'Tourist attraction'] Popular surfing beach offering massive wintertime waves, with calmer waters in the summer.
---


## 5. Preprocess Corpus for LDA

In [17]:
import nltk 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):    
    tokens = word_tokenize(text.lower())    
    return [lemmatizer.lemmatize(w) for w in tokens if w.isalpha() and w not in stop_words]
processed_corpus = [preprocess_text(doc) for doc in corpus]

[nltk_data] Downloading package punkt to /Users/yumin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/yumin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/yumin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## 6. Create Dictionary and Corpus for LDA

In [20]:
from gensim import corpora
dictionary = corpora.Dictionary(processed_corpus)
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

## 7. Train the LDA Model

In [27]:
from gensim.models import LdaModel
lda_model = LdaModel(bow_corpus, num_topics=100, id2word=dictionary, passes=15)
for idx, topic in lda_model.print_topics(-1):    print(f'Topic: {idx} Words: {topic}')

Topic: 0 Words: 0.213*"getting" + 0.117*"job" + 0.083*"bill" + 0.052*"beauty" + 0.052*"crew" + 0.047*"traffic" + 0.036*"e" + 0.028*"blowing" + 0.017*"mine" + 0.016*"celebrating"
Topic: 1 Words: 0.252*"small" + 0.104*"stand" + 0.070*"stadium" + 0.069*"meet" + 0.061*"charge" + 0.059*"swap" + 0.055*"biggest" + 0.054*"aloha" + 0.053*"state" + 0.052*"admission"
Topic: 2 Words: 0.350*"ice" + 0.217*"cream" + 0.084*"enjoy" + 0.051*"sand" + 0.049*"frozen" + 0.028*"sweet" + 0.022*"trying" + 0.017*"swim" + 0.016*"horrible" + 0.016*"ambience"
Topic: 3 Words: 0.165*"pretty" + 0.143*"fried" + 0.129*"much" + 0.077*"cook" + 0.058*"server" + 0.051*"eating" + 0.049*"trip" + 0.038*"another" + 0.027*"run" + 0.023*"nearby"
Topic: 4 Words: 0.245*"relaxed" + 0.241*"waikiki" + 0.137*"contemporary" + 0.111*"naval" + 0.094*"prepared" + 0.022*"airy" + 0.019*"produce" + 0.012*"dirty" + 0.012*"cause" + 0.011*"smell"
Topic: 5 Words: 0.197*"trail" + 0.151*"area" + 0.104*"way" + 0.096*"fall" + 0.086*"clean" + 0.081*"

## 8. Visualize the Topics

In [29]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, bow_corpus, dictionary)
vis

/Users/yumin/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=50637) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/yumin/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=50637) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/yumin/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=50637) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/yumin/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=50637) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/yumin/opt/anaconda3/lib/python3.12/site-packages/jobl

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
45    -0.367588 -0.074903       1        1  3.307779
12     0.022838 -0.328298       2        1  3.258174
40    -0.372399 -0.013791       3        1  2.975302
34     0.075180 -0.127450       4        1  2.549471
44    -0.354711 -0.054945       5        1  2.455376
...         ...       ...     ...      ...       ...
35     0.000250  0.063676      96        1  0.442168
84     0.059953  0.064999      97        1  0.440788
22     0.059355  0.062510      98        1  0.423102
68     0.069964  0.094824      99        1  0.328860
0      0.078284  0.115938     100        1  0.188643

[100 rows x 5 columns], topic_info=           Term          Freq         Total  Category  logprob  loglift
255  restaurant  14962.000000  14962.000000   Default  30.0000  30.0000
0         great   2946.000000   2946.000000   Default  29.0000  29.0000
260        food   4187.000000   4187.000000   Default  28.0000  28.0000
917       store   1953.000000   1953.000000   Default  27.0000  27.0000
72         good   2714.000000   2714.000000   Default  26.0000  26.0000
..          ...           ...           ...       ...      ...      ...
9        summer      0.015747     24.809863  Topic100 -10.3408  -1.0893
10      surfing      0.015747     60.088220  Topic100 -10.3408  -1.9738
11        water      0.015747    558.638999  Topic100 -10.3408  -4.2035
12         wave      0.015747    277.252879  Topic100 -10.3408  -3.5030
13   wintertime      0.015747     13.428386  Topic100 -10.3408  -0.4754

[3995 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1121     89  0.406438   abandoned
223      42  0.805831        able
223      55  0.187068        able
2606     99  0.935292    absolute
782      80  0.990235  absolutely
...     ...       ...         ...
905      67  0.989195         yet
2025     73  0.928819        york
2590     58  0.778243     younger
2016     34  0.975533         yum
1004     26  0.993011       yummy

[2292 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[46, 13, 41, 35, 45, 44, 70, 48, 64, 62, 9, 47, 79, 54, 96, 91, 28, 8, 68, 100, 84, 50, 94, 88, 38, 14, 97, 53, 39, 66, 6, 24, 56, 43, 76, 30, 7, 29, 83, 22, 18, 65, 74, 3, 4, 15, 20, 92, 63, 37, 52, 17, 27, 73, 80, 5, 98, 81, 99, 26, 78, 2, 19, 55, 16, 59, 72, 42, 10, 82, 12, 25, 32, 21, 57, 71, 67, 58, 61, 93, 31, 51, 95, 77, 90, 33, 75, 89, 11, 87, 49, 34, 40, 60, 86, 36, 85, 23, 69, 1])

## 9. Compute Similarity Score

In [31]:
import numpy as np 
from scipy.spatial.distance import cosine
def get_lda_vector(text):    
    processed_text = preprocess_text(text)    
    bow_vector = dictionary.doc2bow(processed_text)    
    lda_vector = lda_model.get_document_topics(bow_vector, minimum_probability=0.0)    
    dense_vector = np.zeros(lda_model.num_topics)    
    for topic_num, prop_topic in lda_vector:        
        dense_vector[topic_num] = prop_topic    
    return dense_vector
    
def calculate_cosine_similarity(vec1, vec2):    
    return 1 - cosine(vec1, vec2)
    
# Example with the first review
sample_review = df['review_document'].iloc[0]
sample_business = df['business_document'].iloc[0]
review_vector = get_lda_vector(sample_review)
business_vector = get_lda_vector(sample_business)
similarity_score = calculate_cosine_similarity(review_vector, business_vector)
print(f'Review: {sample_review}')
print(f'Business: {sample_business}')
print(f'Similarity Score (Trustworthiness): {similarity_score:.4f}')

Review: great for sunset
Business: 'Ehukai Beach Park ['Park', 'Public beach', 'Tourist attraction'] Popular surfing beach offering massive wintertime waves, with calmer waters in the summer.
Similarity Score (Trustworthiness): 0.0122
